In [1]:
%load_ext autoreload
%autoreload 2

### External Imports

In [2]:
import os
import torch
import torchvision as tv
import random
import cv2 as cv
import numpy as np
import typing as t
import pathlib as pb
from ultralytics import YOLO

### Internal Imports

In [3]:
from res import Assets
from alg import VehicleOccupancy, VehicleTracking
from data import CarTrafficDataset, CarTrafficTaskOneDict, CarTrafficTaskTwoDict

### Environment Setup

In [4]:
PATH_ROOT: pb.Path = pb.Path('..')
PATH_ASSETS: pb.Path = PATH_ROOT / 'assets'
PATH_MASKS: pb.Path = PATH_ASSETS / 'masks'
PATH_MODELS: pb.Path = PATH_ASSETS / 'models'
PATH_DATA: pb.Path = PATH_ROOT / 'data' / 'test' # < ---- Place data under this folder
                                                 # test/Task1
                                                 # test/Task2
                                                 # test/Task3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
SEED: int = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
np.random.default_rng(SEED)
torch.set_grad_enabled(False)

### Data Loading

In [6]:
dataset: CarTrafficDataset = CarTrafficDataset(PATH_DATA)
assets: Assets = Assets(PATH_ASSETS)
debug: bool = False

### Task 1

In [7]:
# Create the task solver
vo = VehicleOccupancy(assets, device, debug=debug)

# Solve each example in the task subset
for i in range(len(dataset.task_1)):
    data: CarTrafficTaskOneDict = dataset.task_1[i]
    output: t.List[int] = vo(data)
    output_file: pb.Path = dataset.task_1.path / f"{data['context']:02}_{data['example']}_predicted.txt"

    # Write the answers
    with open(output_file, 'w') as out_file:
        out_file.write(str(len(output)) + os.linesep)
        for i, (query, answer) in enumerate(zip(data['query'], output)):
            out_file.write(str(query) + ' ' + str(answer))
            if i < len(output) - 1:
                out_file.write(os.linesep)

/home/invokariman/.cache/pypoetry/virtualenvs/traffic-monitoring-ys_hD-I5-py3.10/lib/python3.10/site-packages/torch/nn/modules/conv.py:459: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


### Task 2

In [8]:
# Iterate through each video in the dataset
for i in range(0, len(dataset.task_2)):
    # Create task solver for each video
    context: CarTrafficTaskTwoDict = next(iter(dataset.task_2[i]))
    tracks: t.List[t.Tuple[int, int, int, int]] = []
    vt = VehicleTracking(assets, device, debug=debug)

    # Track the vehicle in each frame of the video
    for data in dataset.task_2[i]:
        # Compute BBOXes
        bbox: t.Tuple[int, int, int, int] | None = vt(data)

        # If the car is outside then stop
        if bbox is None:
            break

        # Draw
        image: np.ndarray = data['image'].copy()
        image: np.ndarray = cv.rectangle(image, (bbox[0], bbox[1], bbox[2] - bbox[0], bbox[3] - bbox[1]), color=(0, 0, 255), thickness=5)
        tracks.append(bbox)
        context = data

    # Write the answers
    output_file: pb.Path = dataset.task_2.path / f"{context['context']:02}_predicted.txt"
    with open(output_file, 'w') as out_file:
        out_file.write(f"{context['total']} -1 -1 -1 -1" + os.linesep)
        for j, (x, y, x_, y_) in enumerate(tracks):
            out_file.write(f'{j} {x} {y} {x_} {y_}' + os.linesep)


0: 320x640 17 cars, 54.0ms
Speed: 1.9ms preprocess, 54.0ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 17 cars, 2.5ms
Speed: 0.9ms preprocess, 2.5ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 18 cars, 2.5ms
Speed: 1.0ms preprocess, 2.5ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 18 cars, 2.5ms
Speed: 0.9ms preprocess, 2.5ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 18 cars, 2.8ms
Speed: 0.9ms preprocess, 2.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 18 cars, 2.5ms
Speed: 0.9ms preprocess, 2.5ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 18 cars, 2.5ms
Speed: 1.0ms preprocess, 2.5ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 18 cars, 2.4ms
Speed: 0.8ms preprocess, 2.4ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 16

### Task 3